In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Local/dcgan"

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Local/dcgan
Found GPU at: /device:GPU:0


Comment out the above if not using Google Colaboratory. 

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np

Using TensorFlow backend.


In [0]:
class DCGAN(): 
  def __init__(self, imgrows=28, imgcols=28, channels= 1, latent_dim= 100):
    self.img_rows= imgrows
    self.img_cols= imgcols
    self.channels= channels
    self.img_shape= (self.img_rows, self.img_cols, self.channels)
    self.latent_dim= latent_dim
    
    #parameters chosen from the paper by Radford et al.
    optimizer= Adam(0.0002, 0.5)
    
    #construct generator
    self.generator= self.generator()
    
    #construct discriminator
    self.discriminator= self.discriminator()
    self.discriminator.compile(loss= "binary_crossentropy",
                              optimizer= optimizer, 
                              metrics= ['accuracy'])
    
    #generator generates images from noise
    noise= Input(shape= (self.latent_dim, ))
    img= self.generator(noise)
    
    #only train generator
    self.discriminator.trainable= False
    
    #validity of image as determined by discriminator
    val= self.discriminator(img)
    
    #stack generator and discriminator
    self.GAN = Model(noise, val)
    self.GAN.compile(loss= "binary_crossentropy", optimizer= optimizer)
    
  
  def generator(self): 
    model = Sequential()
    model.add(Dense(128 * 7 * 7, activation= "relu", input_dim= self.latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size= 3, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size= 3, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(self.channels, kernel_size= 3, padding= "same"))
    model.add(Activation("tanh"))
    print("Generator Summary ... ")
    model.summary()
    
    #input noise into generator
    noise= Input(shape= (self.latent_dim, ))
    #pass noise through model to retrieve image
    img= model(noise)
    
    return Model(noise, img)
  
  def discriminator(self): 
    model= Sequential()
    model.add(Conv2D(32, kernel_size= 3, strides= 2, input_shape= self.img_shape, padding= "same"))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size= 3, strides= 2, padding= "same"))
    model.add(ZeroPadding2D(padding= ((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size= 3, strides= 2, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides= 1, padding= "same"))
    model.add(BatchNormalization(momentum= 0.8))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation= "sigmoid"))
    print("Discriminator Summary ... ")
    model.summary()
    
    img= Input(shape= self.img_shape)
    val= model(img)
    return Model(img, val)
  
  def images(self, name= "untitled", show= False): 
    rows, cols= 5, 5
    noise= np.random.normal(0, 1, (rows * cols, self.latent_dim))
    generated= self.generator.predict(noise)
    
    #rescale images to [0, 1]
    generated= 0.5* generated + 0.5
    
    fig, axs= plt.subplots(rows, cols)
    count= 0
    for row in range(rows): 
      for col in range(cols): 
        axs[row, col].imshow(generated[count, :, :, 0], cmap= "gray")
        axs[row, col].axis("off")
        count += 1
    
    fig.savefig("images/%s.png" % name)
    if not show: 
      plt.close()
      
  def train(self, images, epochs= 4000, batch_size= 32, save_interval= 50):
    images= images / 127.5 - 1
    images= np.expand_dims(images, axis= 3)

    #adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
      #train discriminator....
      #select random images
      imgs= images[np.random.randint(0, images.shape[0], batch_size)]

      #create random noise and generate new images
      noise= np.random.normal(0, 1, (batch_size, self.latent_dim))
      generated= self.generator.predict(noise)

      d_loss_real= self.discriminator.train_on_batch(imgs, valid)
      d_loss_fake= self.discriminator.train_on_batch(generated, fake)
      d_loss= 0.5* np.add(d_loss_real, d_loss_fake)
      
      #train generator, by creating images that seem like valid ones
      g_loss= self.GAN.train_on_batch(noise, valid)
      
      print("[%d] D loss: %f, acc: %.2f%% || G loss: %f" %
            (epoch, d_loss[0], d_loss[1] * 100, g_loss))
      
      if epoch % save_interval == 0:
        name= "mnist_%d" % epoch
        self.images(name= name)
    

In [4]:
(X_train, _), (_, _) = mnist.load_data()
dcgan= DCGAN()

11493376/11490434 [==============================] - 1s 0us/step
Generator Summary ... 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
______________________________________________________

In [5]:
dcgan.train(images= X_train)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] D loss: 1.092648, acc: 26.56% || G loss: 0.541473
[1] D loss: 0.863122, acc: 54.69% || G loss: 0.852563
[2] D loss: 0.661522, acc: 59.38% || G loss: 1.233294
[3] D loss: 0.593231, acc: 65.62% || G loss: 1.394157
[4] D loss: 0.822789, acc: 54.69% || G loss: 1.635609
[5] D loss: 0.780301, acc: 51.56% || G loss: 1.510872
[6] D loss: 0.601189, acc: 64.06% || G loss: 1.385490
[7] D loss: 0.570170, acc: 70.31% || G loss: 1.321834
[8] D loss: 0.520352, acc: 70.31% || G loss: 1.257237
[9] D loss: 0.488370, acc: 75.00% || G loss: 0.859698
[10] D loss: 0.706953, acc: 62.50% || G loss: 1.711391
[11] D loss: 0.592807, acc: 68.75% || G loss: 1.421819
[12] D loss: 0.616962, acc: 62.50% || G loss: 1.363261
[13] D loss: 0.646535, acc: 64.06% || G loss: 1.589082
[14] D loss: 0.716824, acc: 60.94% || G loss: 1.779551
[15] D loss: 1.013050, acc: 46.88% || G loss: 1.355668
[16] D loss: 0.726738, acc: 59.38% || G loss: 1.755579
[17] D loss: 0.843591, acc: 50.00% || G loss: 1.500094
[18] D loss: 0.82823